In [1]:
from transformers import BertTokenizer, TFBertModel
from pymongo import MongoClient, ASCENDING, DESCENDING, HASHED
from tqdm.notebook import tqdm

conn = MongoClient()

fake_fibvid = conn.fake_fibvid
tweets  = fake_fibvid.tweets

process_fibvid = conn.process_fibvid
tweets_tokens  = process_fibvid.tweets_tokens

tweets_tokens.create_index([('num_tokens', ASCENDING)])
tweets_tokens.create_index([('created', ASCENDING)])
tweets_tokens.create_index([('tweetId', HASHED)])
tweets_tokens.create_index([('userId', HASHED)])

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [3]:
total = tweets.count_documents({})

to_store = []

for tweet in tqdm(tweets.find(), total=total):
    tokens = tokenizer(tweet['text'], truncation=True)
    tokens['num_tokens'] = len(tokens['input_ids'])
    tokens['created'] = tweet['created']
    tokens['userId'] = tweet['userId']
    tokens['tweetId'] = tweet['tweetId']
    to_store.append(tokens)
    if len(to_store) > 10000:
        tweets_tokens.insert_many(to_store)
        to_store = []
        

if len(to_store) > 0:
    tweets_tokens.insert_many(to_store)
    to_store = []

  0%|          | 0/299118 [00:00<?, ?it/s]